<img style="float:left" width="40%" src="pics/Universidad Burgos.png">
<img style="float:right" width="16%" src="pics/person1_bacteria_2.jpeg">

<br style="clear:both;">

# Trabajo Fin de Grado

<h2 style="display: inline-block; padding: 4mm; padding-left: 2em; background-color: navy; line-height: 1.3em; color: white; border-radius: 10px;">NOMBRE TFG</h2>

### Nuria Martínez Queralt

### Grado en Ingeniería de la Salud 


En este notebook se han llevado a cabo una serie de tareas para la realización del TFG, el cual consiste en la identificación de neumonía a partir de radiografías de tórax empleando una red neuronal. Para esto, se deben probar distintos modelos hasta llegar al modelo más optimo de red neuronal para este caso.

## Redistribución de las imágenes

Debido a que la distribución inicial obtenida a partir del dataset descargado de internet: "https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia" incluye únicamente 16 imágenes en la carpeta de validación ("val") y, esto supone problemas para la obtención de buenos resultados a la hora de construir nuestra red neuronal, antes de empezar a trabajar con las imágenes, se debe crear una función para obtener un nuevo dataset con nuevas carpetas "train", "test" y "val" y una nueva distribución de las imágenes.

In [ ]:
import os

def buscar_imagen(directorio_padre, nombre_imagen):
    '''
    Función empleada para encontrar una imagen concreta (a partir de su nombre) dentro de cualquiera de las subcarpetas del directorio_padre.
    ---------------------------------------------------------
    Parámetros:
    - directorio_padre: ruta donde se encuentra la carpeta cada una de las subcarpetas con las imágenes de radiografías de tórax
    - nombre_imagen: nombre de la imágen a la que se desea acceder 
    ----------------------------------------------------------
    Return:
    - ruta_imagen: ruta completa de la imágen a la que se desea acceder 
    '''
    # Subcarpetas principales en las que buscar
    subcarpetas_principales = ['train', 'test', 'val']
    # Subcarpetas adicionales en las que buscar dentro de cada subcarpeta principal
    subcarpetas_adicionales = ['NORMAL', 'PNEUMONIA']

    # Se itera sobre las subcarpetas principales
    for subcarpeta_principal in subcarpetas_principales:
        # Se itera sobre las subcarpetas adicionales dentro de cada subcarpeta principal
        for subcarpeta_adicional in subcarpetas_adicionales:
            # Se obtiene la ruta completa de la imagen
            ruta_imagen = os.path.join(directorio_padre, subcarpeta_principal, subcarpeta_adicional, nombre_imagen)
            # verificar si la imagen existe en la subcarpeta actual
            if os.path.exists(ruta_imagen):
                return ruta_imagen  # devolver la ruta de la imagen si se encuentra

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil

def redestribucion_imagenes(directorio_principal):

    '''
    Función empleada para redestribuir las imágenes ubicadas en distintas subcarpetas dentro de la carpeta data en una carpeta nueva con las
    mismas subcarpetas pero con un porcentaje distinto de imágenes en cada subcarpeta. La distribución quedaría de la siguiente manera:
    - test: 20% del total
    - train: 64% del total
    - val: 16% del total
    De igual forma, la distribución de las carpetas "PNEUMONIA" y "NORMAL" también queda de forma proporcional.
    --------------------------------------------------------------------
    Parámetros:
    - directorio_principal: ruta donde se encuentra la carpeta data con cada una de las subcarpetas con las imágenes de radiografías de tórax
    -------------------------------------------------------------------
    Return: 
    - nada
    '''

    '''
    En primer lugar, se crea un csv con dos columnas nombres_ficheros y clases compuesto por todas las imágenes existentes en el directorio_padre.
    En la columna nombres_ficheros debe aparecer el nombre de TODAS las imágenes que existen dentro de cada subcarpeta y en la columna clases debe 
    aparecer 0 o 1 en función si se trata de una imagen de la carpeta NORMAL o PNEUMONIA respectivamente.
    '''

    directorio_padre = os.path.join(directorio_principal, 'data')
    
    # Listas para almacenar los nombres de las imágenes y las clases (0 o 1 en función de si es normal o neumonía respectivamente)
    nombres_ficheros = []
    clases = []
    
    # Recorremos las carpetas de train, test y val
    for subcarpeta in ['train', 'test', 'val']:
        ruta_subcarpeta = os.path.join(directorio_padre, subcarpeta)
        for clase in ['NORMAL', 'PNEUMONIA']:
            ruta_clase = os.path.join(ruta_subcarpeta, clase)
            for nombre_fichero in os.listdir(ruta_clase):
                #CREO QUE LA SIGUIENTE LINEA NO HACE FALTA
                #if nombre_fichero.endswith(('.png', '.jpg', '.jpeg')):  
                nombres_ficheros.append(nombre_fichero)
                clases.append(0 if clase == 'NORMAL' else 1)
    
    # Se crea el DataFrame con los datos
    df_todas = pd.DataFrame({'nombre_fichero': nombres_ficheros,'clase': clases})
    
    # Se guarda el DataFrame en un archivo CSV
    ruta_csv = os.path.join(directorio_padre, 'dataset_info.csv') #el nuevo dataframe se guarda dentro del directorio padre
    df_todas.to_csv(ruta_csv, index=False)

    '''
    A partir del csv anterior y, con ayuda de la función train_test_split de skitlearn de divide el csv anterior en dos 
    subgrupos de train y test en proporción 80, 20 para poder usar el 80% de las imágenes para train y el 20% para test.
    También se emplea el parámetro stratify para que exista una proporción de clases en cada uno de los grupos, es decir, en ''NORMAL" y "PNEUMONIA".
    '''
    
    # se emplea train_test_split para dividir el dataset en train (80%) y test (20%)
    # random_state=42 se emplea para que cada vez que se ejecute el código, se obtenga la misma división de datos. El valor 42 es un valor que se usa
    # comunmente en este caso pero se puede emplear cualquie otro valor entero
    # stratify se emplea para agrupar de manera proporcional las clases neumonia y normal en los distintos dataframes
    train_df, test_df = train_test_split(df_todas, test_size=0.2, stratify=df_todas['clase'], random_state=42)
    
    # Se guardan los nuevos conjuntos de datos en archivos CSV
    ruta_train_csv = os.path.join(directorio_padre, 'train_dataset_info.csv') #el nuevo dataframe se guarda dentro del directorio padre
    ruta_test_csv = os.path.join(directorio_padre, 'test_dataset_info.csv') #el nuevo dataframe se guarda dentro del directorio padre
    train_df.to_csv(ruta_train_csv, index=False)
    test_df.to_csv(ruta_test_csv, index=False)

    '''
    A continuación, se coge el conjunto de datos obtenido previamente de train, es decir, el csv "train_df" y se repite el mismo
    proceso pero, esta vez dividiendo este conjunto de datos para train y val en un 80% y 20% respectivamente.
    De tal forma que, finalemnte se obtenga el conjunto de test que represeneta el 20% del total (obtenido previamente), el conjunto de train
    que representa el 80% del 80% del total ya que, inicialmente nos hemos quedado con el 80% pero luego, de este 80%, el 20% va destinado al conjunto
    de validación. Por lo que finalmete quedarías distribuidos de la siguiente manera:
    - test: 20% del total
    - train: 64% del total
    - val: 16% del total
    '''

    # Se emplea train_test_split para dividir el conjunto de datos de entrenamiento en train (80%) y val (20%)
    train_def_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['clase'], random_state=42)
    
    # Se guardan los nuevos conjuntos de datos en archivos CSV
    ruta_train_final_csv = os.path.join(directorio_padre, 'train_final_dataset_info.csv') #el nuevo dataframe se guarda dentro del directorio padre
    ruta_val_csv = os.path.join(directorio_padre, 'val_dataset_info.csv') #el nuevo dataframe se guarda dentro del directorio padre
    train_def_df.to_csv(ruta_train_final_csv, index=False)
    val_df.to_csv(ruta_val_csv, index=False)

    '''
    Finalmente, se crea una nueva carpeta denominada data_nuevo dentro del directorio principal. Dentro de esta carpeta se crean 3 subcarpetas 
    ("train", "test" y "val") que corresponderian con los dataframes obtenidos hasta hora: train_def_df, val_df y test_df y, dentro de estas 3 
    subcarpetas, se crean 2 carpetas "NORMAL" y "PNEUMONIA" que corresponden con con las clases determinadas en cada dataframe, 0 en caso de 
    "NORMAL" y 1 para "PNEUMONIA". Dentro de estas dos carpetas para ("train", "test" y "val") se encontraran las imagenes correspondientes 
    para cada caso según los dataframes obtenidos.
    '''

    # Se crea la nueva carpeta dentro del directorio principal
    ruta_principal_nueva = os.path.join(directorio_principal, 'data_nuevo') 

    # Se crean las carpetas 'train', 'test' y 'val' dentro de la nueva carpeta principal
    for subcarpeta in ['train', 'test', 'val']:
        ruta_subcarpeta = os.path.join(ruta_principal_nueva, subcarpeta)
        os.makedirs(ruta_subcarpeta, exist_ok=True) #verifica si la carpeta ruta_subcarpeta ya existe. Si existe, no se hace nada y el programa continúa su ejecución sin lanzar un error. Si no existe, la función os.makedirs() la crea junto con cualquier carpeta intermedia necesaria en la ruta especificada
        
        # Se crean las subcarpetas 'normal' y 'neumonia' dentro de cada subcarpeta ('train', 'test' y 'val')
        for clase in ['NORMAL', 'PNEUMONIA']:
            ruta_clase = os.path.join(ruta_subcarpeta, clase)
            os.makedirs(ruta_clase, exist_ok=True)
    
                
    # Se copian los archivos CSV a las subcarpetas correspondientes
    for df, nombre_carpeta in [(train_def_df, 'train'), (val_df, 'val'), (test_df, 'test')]:
        for index, row in df.iterrows(): #se itera sobre cada dataframe fila a fila
            clase = 'NORMAL' if row['clase'] == 0 else 'PNEUMONIA'
            nombre_archivo = row['nombre_fichero']
    
            # ruta de origen donde se busca la imagen concreta a partir de la función realizada previamente
            # esta ruta se refiere a donde esta que se desea guardar en la carpeta destino originalmente para poder copiarla
            ruta_origen=buscar_imagen(directorio_padre, nombre_archivo)
            
            # ruta donde se desa guardar (y redestribuir de la forma correcta) las imágenes
            ruta_destino = os.path.join(ruta_principal_nueva, nombre_carpeta, clase, nombre_archivo)
            
            shutil.copyfile(ruta_origen, ruta_destino) # copia las imágenes de la ruta incial a la ruta final
    

In [ ]:
directorio_principal = 'C:/Users/nuria/Downloads/TFG' #ruta donde se encuentra la carpeta data en mi caso y donde se va a crear la nueva carpeta data_nuevo
redestribucion_imagenes(directorio_principal)

In [ ]:
#BORRA TODO LO QUE ESTA A CONTINUACIÓN UNA VEZ SE COMPRUEBE QUE LA FUNCIÓN FUNCIONA CORRECTAMNTE

In [ ]:
'''
Se crea un csv con dos columnas nombres_ficheros y clases. En la columna nombres_ficheros debe aparecer el nombre de TODAS 
las imágenes que existen dentro de cada subcarpeta y en la columna clases debe aparecer 0 o 1 en función si se trata de una imágenes 
de una de las carpetas de NORMAL o PNEUMONIA respectivamente
'''
import os
import pandas as pd

# ruta donde se encuentra el dataset con la distribución inicial descargada de internet
directorio_padre='C:/Users/nuria/Downloads/TFG/data'

# Listas para almacenar los nombres de las imágenes y las clases (0 o 1 en función de si es normal o neumonía respectivamente)
nombres_ficheros = []
clases = []

# Recorremos las carpetas de train, test y val
for subcarpeta in ['train', 'test', 'val']:
    ruta_subcarpeta = os.path.join(directorio_padre, subcarpeta)
    for clase in ['NORMAL', 'PNEUMONIA']:
        ruta_clase = os.path.join(ruta_subcarpeta, clase)
        for nombre_fichero in os.listdir(ruta_clase):
            #CREO QUE LA SIGUIENTE LINEA NO HACE FALTA
            #if nombre_fichero.endswith(('.png', '.jpg', '.jpeg')):  # Puedes ajustar esto a tus extensiones de imagen
            nombres_ficheros.append(nombre_fichero)
            clases.append(0 if clase == 'NORMAL' else 1)

# Se crea el DataFrame con los datos
df_todas = pd.DataFrame({'nombre_fichero': nombres_ficheros,'clase': clases})

# Se guarda el DataFrame en un archivo CSV
ruta_csv = os.path.join(directorio_padre, 'dataset_info.csv')
df_todas.to_csv(ruta_csv, index=False)

print(f'Archivo CSV guardado en: {ruta_csv}')


In [ ]:
df_todas

In [ ]:
#DENTRO DE LAS CARPETAS PNEUMONIA, SE PUEDE TENER EN CUENTA VIRAL Y BACTERIANA

In [ ]:
'''
A partir del csv anterior y, con ayuda de la función train_test_split de skitlearn de debe dividir el csv anterior en dos 
subgrupos de train y test en proporción 80, 20 para poder usar el 80% de las imágenes para train y el 20% para test.
También se emplea el parámetro stratify para que también exista una proporción de clases en cada uno de los grupos.
Posteriormente se deberá dividir el conjunto de train en 2 para obtener así el subconjunto de validación.
'''

import pandas as pd
from sklearn.model_selection import train_test_split

# se emplea train_test_split para dividir el dataset en train (80%) y test (20%)
# random_state=42 se emplea para que cada vez que se ejecute el código, se obtenga la misma división de datos. El valor 42 es un valor que se usa
# comunmente en este caso pero se puede emplear cualquie otro valor entero
# stratify se emplea para agrupar de manera proporcional las clases neumonia y normal en los distintos dataframes
train_df, test_df = train_test_split(df_todas, test_size=0.2, stratify=df_todas['clase'], random_state=42)

# Se guardan los nuevos conjuntos de datos en archivos CSV
ruta_train_csv = 'C:/Users/nuria/Downloads/TFG/data/train_dataset_info.csv' #cambiar ruta en caso necesario (lo ultimo es el nombre del nuevo)
ruta_test_csv = 'C:/Users/nuria/Downloads/TFG/data/test_dataset_info.csv' #cambiar ruta en caso necesario (lo ultimo es el nombre del nuevo)
train_df.to_csv(ruta_train_csv, index=False)
test_df.to_csv(ruta_test_csv, index=False)

print(f'Archivo CSV de entrenamiento guardado en: {ruta_train_csv}')
print(f'Archivo CSV de prueba guardado en: {ruta_test_csv}')


In [ ]:
train_df

In [ ]:
test_df

In [ ]:
'''
A continuación, se coge el conjunto de datos obtenido previamente de train, es decir, el csv "train_df" y se repite el mismo
proceso pero, esta vez dividiendo este conjunto de datos para train y val en un 80% y 20% respectivamente.
De tal forma que, finalemnte se obtenga el conjunto de test que represeneta el 20% del total (obtenido previamente), el conjunto de train
que representa el 80% del 80% del total ya que, inicialmente nos hemos quedado con el 80% pero luego, de este 80%, el 20% va destinado al conjunto
de validación. Por lo que finalmete quedarías distribuidos de la siguiente manera:
- test: 20% del total
- train: 64% del total
- val: 16% del total
'''

from sklearn.model_selection import train_test_split

# Se emplea train_test_split para dividir el conjunto de datos de entrenamiento en train (80%) y val (20%)
train_def_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['clase'], random_state=42)

# Se guardan los nuevos conjuntos de datos en archivos CSV
ruta_train_final_csv = 'C:/Users/nuria/Downloads/TFG/data/train_final_dataset_info.csv' #cambiar ruta en caso necesario (lo ultimo es el nombre del nuevo)
ruta_val_csv = 'C:/Users/nuria/Downloads/TFG/data/val_dataset_info.csv' #cambiar ruta en caso necesario (lo ultimo es el nombre del nuevo)
train_def_df.to_csv(ruta_train_final_csv, index=False)
val_df.to_csv(ruta_val_csv, index=False)

print(f'Archivo CSV de entrenamiento final guardado en: {ruta_train_final_csv}')
print(f'Archivo CSV de validación guardado en: {ruta_val_csv}')


In [ ]:
train_def_df

In [ ]:
val_df

In [ ]:
'''
Función para encontrar una imagen concreta (a partir de su nombre) dentro de cualquiera de las subcarpetas
Esta función es de gran interés para el apartado que se hace a continuación ya que, para encontrar la ruta_origen de la imagen
no se sabe en que carpeta está concretamente y por tanto es necesario acceder a su ruta a partir de esta función
'''

import os

def buscar_imagen(directorio_padre, nombre_imagen):
    # Subcarpetas principales en las que buscar
    subcarpetas_principales = ['train', 'test', 'val']
    # Subcarpetas adicionales en las que buscar dentro de cada subcarpeta principal
    subcarpetas_adicionales = ['NORMAL', 'PNEUMONIA']

    # Se itera sobre las subcarpetas principales
    for subcarpeta_principal in subcarpetas_principales:
        # Se itera sobre las subcarpetas adicionales dentro de cada subcarpeta principal
        for subcarpeta_adicional in subcarpetas_adicionales:
            # Se obtiene la ruta completa de la imagen
            ruta_imagen = os.path.join(directorio_padre, subcarpeta_principal, subcarpeta_adicional, nombre_imagen)
            # verificar si la imagen existe en la subcarpeta actual
            if os.path.exists(ruta_imagen):
                return ruta_imagen  # devolver la ruta de la imagen si se encuentra
    



In [ ]:
'''
Finalmente, se crea una nueva carpeta denominada data_nuevo, dentro de esta carpeta se crean 3 subcarpetas ("train", "test" y "val")
que corresponderian con los dataframes obtenidos hasta hora: train_def_df, val_df y test_df y, dentro de estas 3 subcarpetas, se crean
2 carpetas "NORMAL" y "PNEUMONIA" que corresponden con con las clases determinadas en cada dataframe, 0 en caso de "NORMAL" y 1 para "PNEUMONIA".
Dentro de estas dos carpetas para ("train", "test" y "val") se encontraran las imagenes correspondientes para cada caso según los dataframes obtenidos.
'''

import os
import shutil

# Ruta principal donde se crearán las nuevas carpetas
ruta_principal_nueva = 'C:/Users/nuria/Downloads/TFG/data_nuevo'


# Se crean las carpetas 'train', 'test' y 'val' dentro de la carpeta principal
for subcarpeta in ['train', 'test', 'val']:
    ruta_subcarpeta = os.path.join(ruta_principal_nueva, subcarpeta)
    os.makedirs(ruta_subcarpeta, exist_ok=True) #verifica si la carpeta ruta_subcarpeta ya existe. Si existe, no se hace nada y el programa continúa su ejecución sin lanzar un error. Si no existe, la función os.makedirs() la crea junto con cualquier carpeta intermedia necesaria en la ruta especificada
    
    # Se crean las subcarpetas 'normal' y 'neumonia' dentro de cada subcarpeta ('train', 'test' y 'val')
    for clase in ['NORMAL', 'PNEUMONIA']:
        ruta_clase = os.path.join(ruta_subcarpeta, clase)
        os.makedirs(ruta_clase, exist_ok=True)

            
# Se copian los archivos CSV a las subcarpetas correspondientes
for df, nombre_carpeta in [(train_def_df, 'train'), (val_df, 'val'), (test_df, 'test')]:
    for index, row in df.iterrows(): #se itera sobre cada dataframe fila a fila
        clase = 'NORMAL' if row['clase'] == 0 else 'PNEUMONIA'
        nombre_archivo = row['nombre_fichero']

        # ruta de origen donde se busca la imagen concreta a partir de la función realizada previamente
        # esta ruta se refiere a donde esta que se desea guardar en la carpeta destino originalmente para poder copiarla
        ruta_origen=buscar_imagen('C:/Users/nuria/Downloads/TFG/data', nombre_archivo)
        
        # ruta donde se desa guardar (y redestribuir de la forma correcta) las imágenes
        ruta_destino = os.path.join(ruta_principal_nueva, nombre_carpeta, clase, nombre_archivo)
        
        shutil.copyfile(ruta_origen, ruta_destino) # copia las imágenes de la ruta incial a la ruta final




A partir de aqui, se va a trabajar con la nueva carpeta de imágenes y su nueva distribución para evitar errores

## Preparación del modelo

Se prepara el modelo para poder trabajar con las imágenes de train, test y val

In [1]:
#https://www.kaggle.com/code/paola311/clasificaci-n-de-im-genes-cnn

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import keras
from keras import layers

def preparar_modelo(ruta, batch_size):

    '''
    Función que configura los generadores de datos para entrenar, validar y probar un modelo de aprendizaje automático con imágenes.
    -----------------------------------------------------------
    Parámetros:
    - ruta: str. Ruta base donde se encuentran las imágenes organizadas en subcarpetas (train, val, test)
    - batchsize: int. Tamaño del lote que se utiliza en una única iteración del algoritmo de aprendizaje
    ----------------------------------------------------
    Return:
    - nada
    '''
    
    dir_general = ruta #ubicacion donde se encuentran las imágenes organizadas en subcarpetas (train, val, test). Añadir esta carpeta a one drive en TFG

    dir_train = os.path.join(dir_general, 'train')
    dir_validation = os.path.join(dir_general, 'val')
    dir_test = os.path.join(dir_general, 'test')
    
    # Preprocesamiento de imágenes
    train_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
    validation_datagen=ImageDataGenerator(rescale=1./255)
    
    #Iterador que recorre el directorio de imágenes
    train_generator = train_datagen.flow_from_directory(
        dir_train,
        target_size=(340, 340), #cambiar a (150,150) si no se usa como AlexNet
        batch_size=batch_size, #lo más grande posible que no cause problemas de memoria 
        class_mode='binary')
    
    validation_generator = validation_datagen.flow_from_directory(
        dir_validation,
        target_size=(340, 340), #cambiar a (150,150) si no se usa como AlexNet
        batch_size=batch_size, #lo más grande posible que no cause problemas de memoria 
        class_mode='binary')
    
    test_generator = test_datagen.flow_from_directory(
        dir_test,
        target_size=(340, 340), #cambiar a (150,150) si no se usa como AlexNet
        batch_size=batch_size, #lo más grande posible que no cause problemas de memoria 
        class_mode='binary')
    
    return train_generator, validation_generator, test_generator


    


In [ ]:
ruta='C:/Users/nuria/Downloads/TFG/data_nuevo'
batch_size=20 #ejemplo de batch size

train_generator, validation_generator, test_generator = preparar_modelo(ruta, batch_size)

## Matriz de confusión para ver como funciona el modelo más simple

Se obtiene la matriz de confusión para un primer modelo muy simple para, así poder comprobar como estos resultados mejoran al introducir capas ocultas, modificar parámetros...

In [ ]:
#se trabaja con el modelo más simple (posteriormente denominado Simple1)

input_shape=(340,340,3)

model = keras.Sequential( 
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"), 
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5), 
        layers.Dense(1, activation="sigmoid"), #una unica neurona, sigmoide
    ]
)

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

epochs = 20

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy","Recall","AUC"]) #cambias loss

# con callbacks se detiene el entrenamiento si la pérdida en el conjunto de validación no mejora después de 5 épocas (patience)
model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=EarlyStopping(monitor='val_auc', patience=10,restore_best_weights=True)) 

In [ ]:
y_test=test_generator.labels
y_pred=model.predict(test_generator)

In [ ]:
y_pred=y_pred>0.5 #para convertirlo en un problema binario

In [ ]:
#matriz de confusión con sklearn
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred) #.ravel()

In [ ]:
matriz

In [ ]:
#PERCEPTRON SKLEARN

import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np 

labels=np.unique(y_test)

matriz_conf = metrics.confusion_matrix(y_test, y_pred,labels=labels)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = matriz_conf, display_labels = ["neumonía" , "no neumonía"])
fig, ax = plt.subplots(figsize=(5,5))
cm_display.plot(ax=ax)
plt.title("Matriz de confusión neumonía-no neumonía")
plt.show()

Como se puede observar, los resultados en este primer modelo tan simple, sin ninguna capa oculta, son bastante malos ya que, poniendo esto en un caso clínico real, significaría que 230 pacientes con neumonía hubieran sido diagnosticados como no neumonía y 246 pacientes sin neumonía hubieran sido diagnosticados con neumonía, lo que supondría serios problemas.

## Creación de métricas

Se crea una función para calcular las distintas métricas que servirán para la posterior evaluación de cada modelo que se realice.

In [2]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

'''import numpy as np
import tensorflow as tf'''

def metricas(y_test, y_pred):
    '''
    Funcicón que calcula distintas métricas para la evaluación del modelo.
    -----------------------------------------------------
    Parámetros: 
    - y_test: array de etiquetas verdaderas del conjunto de prueba
    - y_pred: array de etiquetas predichas por el modelo
    ----------------------------------------
    Return: 
    - accuracy: float que indica la proporción de predicciones correctas
    - precision: float que indica la proporción de predicciones positivas correctas
    - recall: float que indica la proporción de positivos detectados
    - f1: float que indica la media armónica de precisión y exhaustividad para evaluar de una forma más equilibrada el rendimiento del modelo
    - specificity: float que indica la proporción de negativos detectados
    - fpr: float que indica la tasa de falsos positivos, es decir, la proporción de negativos incorrectamente clasificadas como positivos, 
    respecto al total de casos negativos reales.
    - fnr: float que indica la tasa de falsos negativos, es decir, la proporción de positivos incorrectamente clasificadas como negativos, 
    respecto al total de casos positivos reales.
    - auc: float que se emplea para evaluar la capacidad de distinción entre clases positivas y negativas de un modelo de clasificación 
    binaria. Un 1 significa que es capaz de distinguir perfectamente entre clases, un 0.5 significa una clasificación aleatoria y un 0 indica 
    que ninguna clase ha sido correctamente clasificada.
    '''
    y_pred_bin=y_pred>0.5 #para convertirlo en un problema binario
    
    #se obtienen los verdaderos negativos, falsos positivos, falsos negativos y verdaderos positivos a partir de la matriz de confusión 
    #con .ravel() se convierte la matriz en un array unidimensional
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_bin).ravel() 

    #se calculan cada una de las métricas empleando su correspondiente fórmula
    accuracy = (tp + tn)/(tn + fp + fn + tp)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * ((precision*recall)/(precision+recall))
    specificity = tn / (tn + fp)
    fpr = fp / (fp + tn) #tasa de falsos positivos
    fnr = fn / (fn + tp) #tasa de falsos negativos
    auc = roc_auc_score(y_test, y_pred)

    
    return [accuracy, precision, recall, f1, specificity, fpr, fnr, auc] #se devuleve como una lista para poder trabajar correctmante con las métricas


    


In [ ]:
y_test=test_generator.labels
y_pred=model.predict(test_generator)

In [ ]:
metricas(y_test, y_pred)

## Comparación de distintas arquitecturas de modelo y distintos batch_size

Para realizar una comparación entre distintas arquitecturas y distintos batch_size, en primer lugar, se generan diferentes modelos de arquitectura de red neuronal variando las capas, el número de capas, etc y, después se entrenan y evalúan los modelos generados con distintos batch sizes. Para esto, se emplean las métricas previamente definidas.

In [ ]:

def establecer_arquitectura(tipo):

    '''
    Función que establece distintos tipos de modelos de red neuronal convolucional (CNN) según el tipo que se introduzca como parámetro.
    --------------------------------------------------------------
    Parámetros
    - tipo: str que indica el tipo de modelo al que se quiere acceder 
    -------------------------------------------------------------
    Return
    -model: modelo sequencial en keras según el tipo que se haya introducido como parámetro de entrada y que contiene toda la información necesaria 
    sobre la arquitectura del modelo
    '''
    
    input_shape=(150,150,3) # se define el tamaño de entrada de las imágenes

    '''
    El modelo Simple1, se corresponde con un modelo que posee varias capas convolucionales (con las que se obtienen características importantes
    de las imágenes) seguidas de capas de MaxPooling2D para reducir la dimensionalidad. Después del Flatten se encuentra una capa densa.
    La función de activación sigmoide en la capa de salida produce una probabilidad entre 0 y 1 para la clasificación binaria.
    Este modelo es muy simple y los resultados no van a ser buenos.
    '''

    
    
    if tipo == "Simple1":
        model = keras.Sequential(
            [
                keras.Input(shape=input_shape),
                layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Flatten(), #convierte imágenes en vectores
                layers.Dropout(0.2), #cambiar a menos de 0,5 
                layers.Dense(1, activation="sigmoid"), #produce una probabilidad entre 0 y 1 para la clasificación binaria
            ]
        )
        
        '''
    El modelo Simple2, se corresponde con un modelo que posee varias capas convolucionales (con las que se obtienen características importantes
    de las imágenes) seguidas de capas de MaxPooling2D para reducir la dimensionalidad. Después del Flatten se encuentra una capa oculta de 
    100 unidades y una capa densa.
    La función de activación sigmoide en la capa de salida produce una probabilidad entre 0 y 1 para la clasificación binaria.
    '''

    elif tipo == "Simple2":
        model = keras.Sequential(
            [
                keras.Input(shape=input_shape),
                layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Flatten(), #convierte imágenes en vectores
                layers.Dense(100, activation="relu"), #100 neuronas en la primera capa
                layers.Dropout(0.2),
                layers.Dense(1, activation="sigmoid"), #produce una probabilidad entre 0 y 1 para la clasificación binaria
            ]
        )
        '''
    El modelo Simple3, se corresponde con un modelo que posee varias capas convolucionales (con las que se obtienen características importantes
    de las imágenes) seguidas de capas de MaxPooling2D para reducir la dimensionalidad. Después del Flatten se encuentra una capa se encuentra 
    una capa oculta de 100 neuronas, una segunda capa oculta de 16 neuronas y una capa densa.
    La función de activación sigmoide en la capa de salida produce una probabilidad entre 0 y 1 para la clasificación binaria.
    '''

    elif tipo == "Simple3":
        model = keras.Sequential(
            [
                keras.Input(shape=input_shape),
                layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Flatten(), #convierte imágenes en vectores
                layers.Dense(100, activation="relu"), #100 neuronas en la primera capa
                layers.Dropout(0.2),
                layers.Dense(16, activation="relu"), #16 neuronas en la segunda capa
                layers.Dropout(0.2),
                layers.Dense(1, activation="sigmoid"), #produce una probabilidad entre 0 y 1 para la clasificación binaria
            ]
        )
    else: #si no se cumple ninguna de las opciones anteriores, aparece un error
        raise ValueError("Tipo de arquitectura no reconocida")
    
    return model #model.summary??




In [ ]:
#prueba con AlexNet
def establecer_arquitectura(tipo):

    '''
    Función que establece distintos tipos de modelos de red neuronal convolucional (CNN) según el tipo que se introduzca como parámetro.
    --------------------------------------------------------------
    Parámetros
    - tipo: str que indica el tipo de modelo al que se quiere acceder 
    -------------------------------------------------------------
    Return
    -model: modelo sequencial en keras según el tipo que se haya introducido como parámetro de entrada y que contiene toda la información necesaria 
    sobre la arquitectura del modelo
    '''
    
    input_shape=(340,340,3) # se define el tamaño de entrada de las imágenes

    '''
    El modelo Simple1, se corresponde con un modelo que posee varias capas convolucionales (con las que se obtienen características importantes
    de las imágenes) seguidas de capas de MaxPooling2D para reducir la dimensionalidad. Después del Flatten se encuentra una capa densa.
    La función de activación sigmoide en la capa de salida produce una probabilidad entre 0 y 1 para la clasificación binaria.
    Este modelo es muy simple y los resultados no van a ser buenos.
    '''

    
    
    if tipo == "Simple1":
        model = keras.Sequential(
            [
                keras.Input(shape=input_shape),
                layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Flatten(), #convierte imágenes en vectores
                layers.Dropout(0.2), #cambiar a menos de 0,5 
                layers.Dense(1, activation="sigmoid"), #produce una probabilidad entre 0 y 1 para la clasificación binaria
            ]
        )
        
        '''
    El modelo Simple2, se corresponde con un modelo que posee varias capas convolucionales (con las que se obtienen características importantes
    de las imágenes) seguidas de capas de MaxPooling2D para reducir la dimensionalidad. Después del Flatten se encuentra una capa oculta de 
    100 unidades y una capa densa.
    La función de activación sigmoide en la capa de salida produce una probabilidad entre 0 y 1 para la clasificación binaria.
    '''

    elif tipo == "Simple2":
        model = keras.Sequential(
            [
                keras.Input(shape=input_shape),
                layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Flatten(), #convierte imágenes en vectores
                layers.Dense(100, activation="relu"), #100 neuronas en la primera capa
                layers.Dropout(0.2),
                layers.Dense(1, activation="sigmoid"), #produce una probabilidad entre 0 y 1 para la clasificación binaria
            ]
        )
        '''
    El modelo Simple3, se corresponde con un modelo que posee varias capas convolucionales (con las que se obtienen características importantes
    de las imágenes) seguidas de capas de MaxPooling2D para reducir la dimensionalidad. Después del Flatten se encuentra una capa se encuentra 
    una capa oculta de 100 neuronas, una segunda capa oculta de 16 neuronas y una capa densa.
    La función de activación sigmoide en la capa de salida produce una probabilidad entre 0 y 1 para la clasificación binaria.
    '''

    elif tipo == "Simple3":
        model = keras.Sequential(
            [
                keras.Input(shape=input_shape),
                layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Flatten(), #convierte imágenes en vectores
                layers.Dense(100, activation="relu"), #100 neuronas en la primera capa
                layers.Dropout(0.2),
                layers.Dense(16, activation="relu"), #16 neuronas en la segunda capa
                layers.Dropout(0.2),
                layers.Dense(1, activation="sigmoid"), #produce una probabilidad entre 0 y 1 para la clasificación binaria
            ]
        )
    else: #si no se cumple ninguna de las opciones anteriores, aparece un error
        raise ValueError("Tipo de arquitectura no reconocida")
    
    return model #model.summary??


In [ ]:
#AlexNet

# Layer 1: Convolutional layer with 96 filters of size 16x16x3
model.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu', input_shape=(340,340,3)))
    
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
    
model.add(BatchNormalization())
    
# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='valid', activation='relu'))
    
# Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
    
model.add(BatchNormalization())


# Layer 3-5: 3 more convolutional layers with similar structure as Layer 1
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
    
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
    
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'))
    
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
    
model.add(BatchNormalization())


# Layer 6: Fully connected layer with 4096 neurons
model.add(layers.Flatten())

model.add(Dense(100, activation="relu"))  
model.add(Dropout(0.2))  
model.add(Dense(1, activation="sigmoid"))  




In [ ]:
#PRUEBA/EJECUTAR 2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import keras
from keras import layers

import pandas as pd
from keras.callbacks import EarlyStopping

def arq_batch(ruta,epochs,batch_sizes,modelos):
    '''
    Función que devuelve una tabla comparativa para distintas arquitecturas de modelo y distintos batch size introducidos como parámetros. 
    ----------------------------------------------------
    Parámetros:
    - ruta: str. Ruta base donde se encuentran las imágenes organizadas en subcarpetas (train, val, test)
    - epochs: int. Número de épocas a entrenar 
    - batch_sizes: lista con los distintos valores de batch size para probar en cada entrenamiento
    - modelos: lista de nombres de cada uno de los modelos que se van a comparar obtenidos partir de la función realizada previamente 
    "establecer_arquitectura(modelo)"
    --------------------------------------------------
    Return:
    - compara_arqu_batch_def: dataframe que contiene como índice las columnas referidas al modelo de arquitectura y al valor de batch size. El dataframe 
    obtenido se observa como una tabla comparativa de diversas métricas para cada arquitectura y cada batch size.
    '''
    
    #se inicializa un dataframe vacío donde, posteriormente se van a añadir todos los componentes necesarios para comparar los distintos 
    #modelos de arquitectura para distintos batch size (comparando las métricas)
    compara_arqu_batch=pd.DataFrame()
    

    #bucle en el que se recorren cada uno de los modelos y los tamaños de batch_size 
    for modelo in modelos:
        print(f"Comparando modelo {modelo}...")
        for batch_size in batch_sizes:
            print(f"Entrenando modelo {modelo} y batch_size {batch_size}")
    
            #se emplea la función preparar_modelo para configurar los generadores de datos para entrenar, validar y probar 
            #un modelo de aprendizaje automático con imágenes
            train_generator, validation_generator, test_generator = preparar_modelo(ruta, batch_size)
            
            #se emplea la función establecer_arquitectura para determinar el modelo con el que se trabaja cada vez
            model = establecer_arquitectura(modelo)
            
            #se compila el modelo y se calculan las métricas con las que se quiere trabajar
            #en este caso, en la función de pérdida "loss", se emplea la entropía cruzada binaria "binary_crossentropy" ya que se trata de 
            #un problema de clasificación binaria
            model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy","Recall","AUC"]) #cambias loss
    
            #ENTRENA
            # con callbacks se detiene el entrenamiento si la pérdida en el conjunto de validación no mejora después de 10 épocas (patience)
            model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=EarlyStopping(monitor='val_auc', patience=10,restore_best_weights=True))
    
            #se calculan las métricas
            y_test=test_generator.labels
            y_pred=model.predict(test_generator)
            calculo_metricas=metricas(y_test, y_pred) #se llama a la función creada previamente para calcular las métricas de cada modelo
            #se calcula loss a partir de la evaluación del modelo
            loss=model.evaluate(test_generator, verbose=0)[0]
            
            #esto es en caso de querer meter todos estos parametros dentro de metricas (cambiando tambien la linea de arriba, en lugar de metricas loss, accuracy...)
            #metricas = f"Loss: {loss}, Accuracy: {accuracy}, Recall: {recall}, AUC: {AUC}, Precision: {precision}"
    
            #cambiar .append por .concat
            #se añaden todos los componentes necesarios para comparar los distintos modelos de arquitectura para distintos batch size 
            #(comparando las métricas)
            compara_arqu_batch=compara_arqu_batch.append({"Red": modelo, "BatchSize": batch_size, "Loss": loss, "Accuracy": calculo_metricas[0], "Precision": calculo_metricas[1], "Recall": calculo_metricas[2], "F1":calculo_metricas[3], "Specificity":calculo_metricas[4], "fpr":calculo_metricas[5], "fnr":calculo_metricas[6], "AUC": calculo_metricas[7]}, ignore_index=True)
    
    #se fijan las columnas Red y BatchSize como índices. 
    compara_arqu_batch.set_index(["Red","BatchSize"], inplace=True) #inplace=True se pone para modificar el dataframe original ya que sino, no se modifica
    compara_arqu_batch_def = compara_arqu_batch.round(2) #se redondean los decimales a 2
    return compara_arqu_batch_def


In [ ]:
#learning rate 0,001 

In [ ]:
ruta='C:/Users/nuria/Downloads/TFG/data_nuevo'
epochs=20
batch_sizes=[8, 16, 20, 32, 64]  # distintos tamaños de batch size para probar
modelos=["Simple1", "Simple2", "Simple3"]  # Lista de nombres de modelos
arq_batch(ruta,epochs,batch_sizes,modelos)

In [ ]:
'''#PRUEBA/EJECUTAR
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import keras
from keras import layers

import pandas as pd
from keras.callbacks import EarlyStopping

batch_sizes=[8, 16, 20, 32, 64]  # distintos tamaños de batch size para probar
modelos=["Simple1", "Simple2", "Simple3"]  # Lista de nombres de modelos

#se inicializa un dataframe vacío donde, posteriormente se van a añadir todos los componentes necesarios para comparar los distintos 
#modelos de arquitectura para distintos batch size (comparando las métricas)
compara_arqu_batch=pd.DataFrame()

epochs=20

#bucle en el que se recorren cada uno de los modelos y los tamaños de batch_size 
for modelo in modelos:
    print(f"Comparando modelo {modelo}...")
    for batch_size in batch_sizes:
        print(f"Entrenando modelo {modelo} y batch_size {batch_size}")

        #SE PREPARA EL MODELO
        dir_general = 'C:/Users/nuria/Downloads/TFG/data' #ubicacion donde yo tengo metida la carpeta data (cambiar en caso necesario) y añadir esta carpeta a one drive en TFG

        dir_train = os.path.join(dir_general, 'train')
        dir_validation = os.path.join(dir_general, 'val')
        dir_test = os.path.join(dir_general, 'test')

        # Preprocesamiento de imágenes
        train_datagen = ImageDataGenerator(rescale=1./255)
        test_datagen = ImageDataGenerator(rescale=1./255)
        validation_datagen=ImageDataGenerator(rescale=1./255)
        
        #Iterador que recorre el directorio de imágenes
        train_generator = train_datagen.flow_from_directory(
            dir_train,
            target_size=(150, 150), #todas las imágenes se redimensionen a 150x150 píxeles, de forma que, si existen tamaños diferentes entre ellas, se uniforman
            batch_size=batch_size, # se itera para distintos valores de batch size
            class_mode='binary')
        
        validation_generator = validation_datagen.flow_from_directory(
            dir_validation,
            target_size=(150, 150), #todas las imágenes se redimensionen a 150x150 píxeles, de forma que, si existen tamaños diferentes entre ellas, se uniforman
            batch_size=batch_size, #lo más grande posible que no cause problemas de memoria 
            class_mode='binary')
        
        test_generator = test_datagen.flow_from_directory(
            dir_test,
            target_size=(150, 150), #todas las imágenes se redimensionen a 150x150 píxeles, de forma que, si existen tamaños diferentes entre ellas, se uniforman
            batch_size=batch_size, #lo más grande posible que no cause problemas de memoria 
            class_mode='binary')
        
        #se emplea la función establecer_arquitectura para determinar el modelo con el que se trabaja cada vez
        model = establecer_arquitectura(modelo)
        
        #se compila el modelo y se calculan las métricas con las que se quiere trabajar
        #en este caso, en la función de pérdida "loss", se emplea la entropía cruzada binaria "binary_crossentropy" ya que se trata de 
        #un problema de clasificación binaria
        model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy","Recall"]) #cambias loss

        #ENTRENA
        # con callbacks se detiene el entrenamiento si la pérdida en el conjunto de validación no mejora después de 3 épocas (patience)
        model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=EarlyStopping(monitor='val_loss', patience=3))

        #se calculan las métricas
        y_test=test_generator.labels
        y_pred=model.predict(test_generator)
        calculo_metricas=metricas(y_test, y_pred) #se llama a la función creada previamente para calcular las métricas de cada modelo
        #se calcula loss a partir de la evaluación del modelo
        loss=model.evaluate(test_generator, verbose=0)[0]
        
        #esto es en caso de querer meter todos estos parametros dentro de metricas (cambiando tambien la linea de arriba, en lugar de metricas loss, accuracy...)
        #metricas = f"Loss: {loss}, Accuracy: {accuracy}, Recall: {recall}, AUC: {AUC}, Precision: {precision}"

        #cambiar .append por .concat
        #se añaden todos los componentes necesarios para comparar los distintos modelos de arquitectura para distintos batch size 
        #(comparando las métricas)
        compara_arqu_batch=compara_arqu_batch.append({"Red": modelo, "BatchSize": batch_size, "Loss": loss, "Accuracy": calculo_metricas[0], "Precision": calculo_metricas[1], "Recall": calculo_metricas[2], "F1":calculo_metricas[3], "Specificity":calculo_metricas[4], "fpr":calculo_metricas[5], "fnr":calculo_metricas[6], "AUC": calculo_metricas[7]}, ignore_index=True)




In [ ]:
'''#se fijan las columnas A y B como índices. 
compara_arqu_batch.set_index(["Red","BatchSize"], inplace=True) #inplace=True se pone para modificar el dataframe original ya que sino, no se modifica
compara_arqu_batch_def = compara_arqu_batch.round(2) #se redondean los decimales a 2
compara_arqu_batch_def

In [ ]:
#Por lo tanto, se puede deducir que, 

In [ ]:
'''import pandas as pd
from keras.callbacks import EarlyStopping

batch_sizes=[8, 16, 20, 32, 64]  # distintos tamaños de batch size para probar
modelos=["Simple1", "Simple2", "Simple3"]  # Lista de nombres de modelos

#se inicializa un dataframe vacío donde, posteriormente se van a añadir todos los componentes necesarios para comparar los distintos 
#modelos de arquitectura para distintos batch size (comparando las métricas)
compara_arqu_batch=pd.DataFrame()

epochs=5

#bucle en el que se recorren cada uno de los modelos y los tamaños de batch_size 
for modelo in modelos:
    print(f"Comparando modelo {modelo}...")
    for batch_size in batch_sizes:
        print(f"Entrenando modelo {modelo} y batch_size {batch_size}")
        
        #se emplea la función establecer_arquitectura para determinar el modelo con el que se trabaja cada vez
        model = establecer_arquitectura(modelo)
        
        #se compila el modelo y se calculan las métricas con las que se quiere trabajar
        #en este caso, en la función de pérdida "loss", se emplea la entropía cruzada binaria "binary_crossentropy" ya que se trata de 
        #un problema de clasificación binaria
        model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy","Recall"]) #cambias loss

        #ENTRENA
        # con callbacks se detiene el entrenamiento si la pérdida en el conjunto de validación no mejora después de 3 épocas (patience)
        model.fit(train_generator, batch_size=batch_size, epochs=epochs, validation_data=validation_generator, callbacks=EarlyStopping(monitor='val_loss', patience=3))

        #se calculan las métricas
        y_test=test_generator.labels
        y_pred=model.predict(test_generator)
        calculo_metricas=metricas(y_test, y_pred) #se llama a la función creada previamente para calcular las métricas de cada modelo
        #se calcula loss a partir de la evaluación del modelo
        loss=model.evaluate(test_generator, verbose=0)[0]

        #esto es en caso de querer meter todos estos parametros dentro de metricas (cambiando tambien la linea de arriba, en lugar de metricas loss, accuracy...)
        #metricas = f"Loss: {loss}, Accuracy: {accuracy}, Recall: {recall}, AUC: {AUC}, Precision: {precision}"

        #cambiar .append por .concat
        #se añaden todos los componentes necesarios para comparar los distintos modelos de arquitectura para distintos batch size 
        #(comparando las métricas)
        compara_arqu_batch=compara_arqu_batch.append({"Red": modelo, "BatchSize": batch_size, "Loss": loss, "Accuracy": calculo_metricas[0], "Precision": calculo_metricas[1], "Recall": calculo_metricas[2], "F1":calculo_metricas[3], "Specificity":calculo_metricas[4], "fpr":calculo_metricas[5], "fnr":calculo_metricas[6], "AUC": calculo_metricas[7]}, ignore_index=True)



In [ ]:
'''compara_arqu_batch

In [ ]:
#Por lo tanto, se puede apreciar que la primera arquitectura es la peor de todas (algo que era de esperar) y la mejor opcion es 
# el Simple 2 ya que, en general (exceptuando para una batch size de 8) obtiene mejores resultados.
#Dentro del Simple 2, el mejor valor de batch size es el de 32 ya que es el que presenta mejores resultados

## Comparación de distintos valores de número de neuronas para la arquitectura "Simple2" y un batchsize de 32

A partir de los resultados obtenidos previamente, se comparan distintos valores de número de neuronas para determinar con cuál funciona mejor el modelo.

In [ ]:
#antes hay que ejecutar la funcion preparar modelo y la funcion de metricas

In [ ]:
#Simple2
import pandas as pd
from keras.callbacks import EarlyStopping

def neuronas(num_neuronas, epochs, ruta, batch_size):

    '''
    Función que devuelve una tabla comparativa para distintas valores de neuronas introducidos como parámetros a partir del modelo y el batch size
    seleccionado previamente.
    ------------------------------------------------------------------------
    Parámetros;
    - num_neuronas:
    - epochs:
    - ruta: str. Ruta base donde se encuentran las imágenes organizadas en subcarpetas (train, val, test)
    - batch_size: int. Tamaño del lote que se utiliza en una única iteración del algoritmo de aprendizaje. Se emplea dentro de la función
    "preparar_modelo" para determinar el tamaño del lote para cada uno de los generadores (train, val y test)
    ----------------------------------------------------------------
    Return:
    - compara_neuronas_def: dataframe que contiene como índice las columnas referidas al número de neuronas. El dataframe 
    obtenido se observa como una tabla comparativa de diversas métricas para cada número de neuronas.
    '''
    
    #se inicializa un dataframe vacío donde, posteriormente se van a añadir todos los componentes necesarios para comparar y determinar cual es el mejor
    #valor de neuronas en la capa oculta
    compara_neuronas=pd.DataFrame()
    
    input_shape=(150,150,3)

    #se emplea la función preparar_modelo para configurar los generadores de datos para entrenar, validar y probar 
    #un modelo de aprendizaje automático con imágenes
    train_generator, validation_generator, test_generator = preparar_modelo(ruta, batch_size)
    
    
    for neurona in num_neuronas:
        print(f"Modelo con {neurona} neuronas en su capa oculta...")

    
        #se emplea el modelo Simple2 que es el que se ha determinado previamente como "mejor"
        model = keras.Sequential(
                [
                    keras.Input(shape=input_shape),
                    layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
                    layers.MaxPooling2D(pool_size=(2, 2)),
                    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
                    layers.MaxPooling2D(pool_size=(2, 2)),
                    layers.Flatten(), #convierte imágenes en vectores
                    layers.Dense(neurona, activation="relu"), #se va cambiando el valor de "neurona" para cada uno de los valores que estan en la lista num_neuronas
                    layers.Dropout(0.2),
                    layers.Dense(1, activation="sigmoid"), #produce una probabilidad entre 0 y 1 para la clasificación binaria
                ]
            )

        
        #se compila el modelo y se calculan las métricas con las que se quiere trabajar
        #en este caso, en la función de pérdida "loss", se emplea la entropía cruzada binaria "binary_crossentropy" ya que se trata de 
        #un problema de clasificación binaria
        model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy","Recall","AUC"]) #cambias loss
    
        #ENTRENA
        # con callbacks se detiene el entrenamiento si la pérdida en el conjunto de validación no mejora después de 10 épocas (patience)
        #se emplea un batch size de 32 que es el que ha dado mejores resultados antes
        model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=EarlyStopping(monitor='val_auc', patience=10,restore_best_weights=True))
    
        #se calculan las métricas
        y_test=test_generator.labels
        y_pred=model.predict(test_generator)
        calculo_metricas=metricas(y_test, y_pred) #se llama a la función creada previamente para calcular las métricas de cada modelo
        #se calcula loss a partir de la evaluación del modelo
        loss=model.evaluate(test_generator, verbose=0)[0]
    
        #esto es en caso de querer meter todos estos parametros dentro de metricas (cambiando tambien la linea de arriba, en lugar de metricas loss, accuracy...)
        #metricas = f"Loss: {loss}, Accuracy: {accuracy}, Recall: {recall}, AUC: {AUC}, Precision: {precision}"
    
        #cambiar .append por .concat
        #se añaden todos los componentes necesarios para comparar los distintos modelos de arquitectura para distintos batch size 
        #(comparando las métricas)
        compara_neuronas=compara_neuronas.append({"Número de neuronas": neurona, "Loss": loss, "Accuracy": calculo_metricas[0], "Precision": calculo_metricas[1], "Recall": calculo_metricas[2], "F1":calculo_metricas[3], "Specificity":calculo_metricas[4], "fpr":calculo_metricas[5], "fnr":calculo_metricas[6], "AUC": calculo_metricas[7]}, ignore_index=True)
    
    #se fija la columna "Número de neuronas" como índice. 
    compara_neuronas.set_index("Número de neuronas", inplace=True) #inplace=True se pone para modificar el dataframe original ya que sino, no se modifica
    compara_neuronas_def = compara_neuronas.round(2) #se redondean los decimales a 2
    return compara_neuronas_def
    
        #PONER int(neurona) si salen como decimanles



In [ ]:
num_neuronas=[512, 1024, 2048] #lista con distintos valores de neuronas para probar
epochs=20
ruta='C:/Users/nuria/Downloads/TFG/data_nuevo'
batch_size=32

neuronas(num_neuronas, epochs, ruta, batch_size)

In [ ]:
#NUMERO DE NEURONAS CON EL MODELO SIMPLE3
import pandas as pd
from keras.callbacks import EarlyStopping

def neuronas(num_neuronas, epochs, ruta, batch_size):

    '''
    Función que devuelve una tabla comparativa para distintas valores de neuronas introducidos como parámetros a partir del modelo y el batch size
    seleccionado previamente.
    ------------------------------------------------------------------------
    Parámetros;
    - num_neuronas:
    - epochs:
    - ruta: str. Ruta base donde se encuentran las imágenes organizadas en subcarpetas (train, val, test)
    - batch_size: int. Tamaño del lote que se utiliza en una única iteración del algoritmo de aprendizaje. Se emplea dentro de la función
    "preparar_modelo" para determinar el tamaño del lote para cada uno de los generadores (train, val y test)
    ----------------------------------------------------------------
    Return:
    - compara_neuronas_def: dataframe que contiene como índice las columnas referidas al número de neuronas. El dataframe 
    obtenido se observa como una tabla comparativa de diversas métricas para cada número de neuronas.
    '''
    
    #se inicializa un dataframe vacío donde, posteriormente se van a añadir todos los componentes necesarios para comparar y determinar cual es el mejor
    #valor de neuronas en la capa oculta
    compara_neuronas=pd.DataFrame()
    
    input_shape=(150,150,3)

    #se emplea la función preparar_modelo para configurar los generadores de datos para entrenar, validar y probar 
    #un modelo de aprendizaje automático con imágenes
    train_generator, validation_generator, test_generator = preparar_modelo(ruta, batch_size)
    
    
    for neurona in num_neuronas:
        print(f"Modelo con {neurona} neuronas en su capa oculta...")
        #se emplea el modelo Simple2 que es el que se ha determinado previamente como "mejor"
        model = keras.Sequential(
            [
                keras.Input(shape=input_shape),
                layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
                layers.MaxPooling2D(pool_size=(2, 2)),
                layers.Flatten(), #convierte imágenes en vectores
                layers.Dense(neurona, activation="relu"), #100 neuronas en la primera capa
                layers.Dropout(0.2),
                layers.Dense(neurona, activation="relu"), #16 neuronas en la segunda capa
                layers.Dropout(0.2),
                layers.Dense(1, activation="sigmoid"), #produce una probabilidad entre 0 y 1 para la clasificación binaria
            ]
        )

        
        #se compila el modelo y se calculan las métricas con las que se quiere trabajar
        #en este caso, en la función de pérdida "loss", se emplea la entropía cruzada binaria "binary_crossentropy" ya que se trata de 
        #un problema de clasificación binaria
        model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy","Recall","AUC"]) #cambias loss
    
        #ENTRENA
        # con callbacks se detiene el entrenamiento si la pérdida en el conjunto de validación no mejora después de 10 épocas (patience)
        #se emplea un batch size de 32 que es el que ha dado mejores resultados antes
        model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=EarlyStopping(monitor='val_auc', patience=10,restore_best_weights=True))
    
        #se calculan las métricas
        y_test=test_generator.labels
        y_pred=model.predict(test_generator)
        calculo_metricas=metricas(y_test, y_pred) #se llama a la función creada previamente para calcular las métricas de cada modelo
        #se calcula loss a partir de la evaluación del modelo
        loss=model.evaluate(test_generator, verbose=0)[0]
        #esto es en caso de querer meter todos estos parametros dentro de metricas (cambiando tambien la linea de arriba, en lugar de metricas loss, accuracy...)
        #metricas = f"Loss: {loss}, Accuracy: {accuracy}, Recall: {recall}, AUC: {AUC}, Precision: {precision}"
    
        #cambiar .append por .concat
        #se añaden todos los componentes necesarios para comparar los distintos modelos de arquitectura para distintos batch size 
        #(comparando las métricas)
        compara_neuronas=compara_neuronas.append({"Número de neuronas": int(neurona), "Loss": loss, "Accuracy": calculo_metricas[0], "Precision": calculo_metricas[1], "Recall": calculo_metricas[2], "F1":calculo_metricas[3], "Specificity":calculo_metricas[4], "fpr":calculo_metricas[5], "fnr":calculo_metricas[6], "AUC": calculo_metricas[7]}, ignore_index=True)
    
    #se fija la columna "Número de neuronas" como índice. 
    compara_neuronas.set_index("Número de neuronas", inplace=True) #inplace=True se pone para modificar el dataframe original ya que sino, no se modifica
    compara_neuronas_def = compara_neuronas.round(2) #se redondean los decimales a 2
    return compara_neuronas_def
    
        #PONER int(neurona) si salen como decimanles


In [ ]:
num_neuronas=[512, 1024, 2048] #lista con distintos valores de neuronas para probar
epochs=20
ruta='C:/Users/nuria/Downloads/TFG/data_nuevo'
batch_size=32

neuronas(num_neuronas, epochs, ruta, batch_size)

In [3]:
#Simple2 AlexNet
import pandas as pd
from keras.callbacks import EarlyStopping

def neuronas(num_neuronas, epochs, ruta, batch_size):

    '''
    Función que devuelve una tabla comparativa para distintas valores de neuronas introducidos como parámetros a partir del modelo y el batch size
    seleccionado previamente.
    ------------------------------------------------------------------------
    Parámetros;
    - num_neuronas:
    - epochs:
    - ruta: str. Ruta base donde se encuentran las imágenes organizadas en subcarpetas (train, val, test)
    - batch_size: int. Tamaño del lote que se utiliza en una única iteración del algoritmo de aprendizaje. Se emplea dentro de la función
    "preparar_modelo" para determinar el tamaño del lote para cada uno de los generadores (train, val y test)
    ----------------------------------------------------------------
    Return:
    - compara_neuronas_def: dataframe que contiene como índice las columnas referidas al número de neuronas. El dataframe 
    obtenido se observa como una tabla comparativa de diversas métricas para cada número de neuronas.
    '''
    
    #se inicializa un dataframe vacío donde, posteriormente se van a añadir todos los componentes necesarios para comparar y determinar cual es el mejor
    #valor de neuronas en la capa oculta
    compara_neuronas=pd.DataFrame()
    
    input_shape=(340,340,3)

    #se emplea la función preparar_modelo para configurar los generadores de datos para entrenar, validar y probar 
    #un modelo de aprendizaje automático con imágenes
    train_generator, validation_generator, test_generator = preparar_modelo(ruta, batch_size)
    for neurona in num_neuronas:
        print(f"Modelo con {neurona} neuronas en su capa oculta...")

    
        #se emplea el modelo Simple2 que es el que se ha determinado previamente como "mejor"
        model = keras.Sequential(
            [
                keras.Input(shape=input_shape),
                layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Flatten(), #convierte imágenes en vectores
                layers.Dense(neurona, activation="relu"), #100 neuronas en la primera capa
                layers.Dropout(0.2),
                layers.Dense(1, activation="sigmoid"), #produce una probabilidad entre 0 y 1 para la clasificación binaria
            ]
        )

        
        #se compila el modelo y se calculan las métricas con las que se quiere trabajar
        #en este caso, en la función de pérdida "loss", se emplea la entropía cruzada binaria "binary_crossentropy" ya que se trata de 
        #un problema de clasificación binaria
        model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy","Recall","AUC"]) #cambias loss
    
        #ENTRENA
        # con callbacks se detiene el entrenamiento si la pérdida en el conjunto de validación no mejora después de 10 épocas (patience)
        #se emplea un batch size de 32 que es el que ha dado mejores resultados antes
        model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=EarlyStopping(monitor='val_auc', patience=10,restore_best_weights=True))
    
        #se calculan las métricas
        y_test=test_generator.labels
        y_pred=model.predict(test_generator)
        calculo_metricas=metricas(y_test, y_pred) #se llama a la función creada previamente para calcular las métricas de cada modelo
        #se calcula loss a partir de la evaluación del modelo
        loss=model.evaluate(test_generator, verbose=0)[0]
    
        #esto es en caso de querer meter todos estos parametros dentro de metricas (cambiando tambien la linea de arriba, en lugar de metricas loss, accuracy...)
        #metricas = f"Loss: {loss}, Accuracy: {accuracy}, Recall: {recall}, AUC: {AUC}, Precision: {precision}"
    
        #cambiar .append por .concat
        #se añaden todos los componentes necesarios para comparar los distintos modelos de arquitectura para distintos batch size 
        #(comparando las métricas)
        compara_neuronas=compara_neuronas.append({"Número de neuronas": int(neurona), "Loss": loss, "Accuracy": calculo_metricas[0], "Precision": calculo_metricas[1], "Recall": calculo_metricas[2], "F1":calculo_metricas[3], "Specificity":calculo_metricas[4], "fpr":calculo_metricas[5], "fnr":calculo_metricas[6], "AUC": calculo_metricas[7]}, ignore_index=True)
    
    #se fija la columna "Número de neuronas" como índice. 
    compara_neuronas.set_index("Número de neuronas", inplace=True) #inplace=True se pone para modificar el dataframe original ya que sino, no se modifica
    compara_neuronas_def = compara_neuronas.round(2) #se redondean los decimales a 2
    return compara_neuronas_def
    
        #PONER int(neurona) si salen como decimanles


In [5]:
num_neuronas=[512, 1024] #lista con distintos valores de neuronas para probar AÑADIR 2048
epochs=20
ruta='C:/Users/nuria/Downloads/TFG/data_nuevo'
batch_size=16

neuronas(num_neuronas, epochs, ruta, batch_size)

Found 3747 images belonging to 2 classes.
Found 937 images belonging to 2 classes.
Found 1172 images belonging to 2 classes.
Modelo con 512 neuronas en su capa oculta...
Epoch 1/20
235/235 [==============================] - 294s 1s/step - loss: 0.4323 - accuracy: 0.8740 - recall: 0.9151 - auc: 0.9140 - val_loss: 0.4703 - val_accuracy: 0.8122 - val_recall: 0.9985 - val_auc: 0.9576
Epoch 2/20
235/235 [==============================] - 283s 1s/step - loss: 0.1973 - accuracy: 0.9247 - recall: 0.9546 - auc: 0.9695 - val_loss: 0.2083 - val_accuracy: 0.9264 - val_recall: 0.9327 - val_auc: 0.9742
Epoch 3/20
235/235 [==============================] - 284s 1s/step - loss: 0.1700 - accuracy: 0.9424 - recall: 0.9609 - auc: 0.9775 - val_loss: 7.2650 - val_accuracy: 0.3799 - val_recall: 0.1506 - val_auc: 0.6798
Epoch 4/20
235/235 [==============================] - 287s 1s/step - loss: 0.1666 - accuracy: 0.9376 - recall: 0.9590 - auc: 0.9780 - val_loss: 0.1513 - val_accuracy: 0.9434 - val_recall: 0.9

C:\Users\nuria\AppData\Local\Temp\ipykernel_2420\1355991842.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compara_neuronas=compara_neuronas.append({"Número de neuronas": int(neurona), "Loss": loss, "Accuracy": calculo_metricas[0], "Precision": calculo_metricas[1], "Recall": calculo_metricas[2], "F1":calculo_metricas[3], "Specificity":calculo_metricas[4], "fpr":calculo_metricas[5], "fnr":calculo_metricas[6], "AUC": calculo_metricas[7]}, ignore_index=True)


Modelo con 1024 neuronas en su capa oculta...
Epoch 1/20
235/235 [==============================] - 307s 1s/step - loss: 0.4463 - accuracy: 0.8759 - recall: 0.9199 - auc: 0.9139 - val_loss: 0.2467 - val_accuracy: 0.9221 - val_recall: 0.9269 - val_auc: 0.9684
Epoch 2/20
235/235 [==============================] - 291s 1s/step - loss: 0.1919 - accuracy: 0.9298 - recall: 0.9557 - auc: 0.9702 - val_loss: 0.1715 - val_accuracy: 0.9413 - val_recall: 0.9503 - val_auc: 0.9773
Epoch 3/20
235/235 [==============================] - 297s 1s/step - loss: 0.2063 - accuracy: 0.9237 - recall: 0.9495 - auc: 0.9683 - val_loss: 0.2140 - val_accuracy: 0.9253 - val_recall: 0.9825 - val_auc: 0.9754
Epoch 4/20
235/235 [==============================] - 296s 1s/step - loss: 0.1553 - accuracy: 0.9394 - recall: 0.9590 - auc: 0.9800 - val_loss: 0.3062 - val_accuracy: 0.8933 - val_recall: 0.9868 - val_auc: 0.9637
Epoch 5/20
235/235 [==============================] - 298s 1s/step - loss: 0.1456 - accuracy: 0.9426 -

C:\Users\nuria\AppData\Local\Temp\ipykernel_2420\1355991842.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compara_neuronas=compara_neuronas.append({"Número de neuronas": int(neurona), "Loss": loss, "Accuracy": calculo_metricas[0], "Precision": calculo_metricas[1], "Recall": calculo_metricas[2], "F1":calculo_metricas[3], "Specificity":calculo_metricas[4], "fpr":calculo_metricas[5], "fnr":calculo_metricas[6], "AUC": calculo_metricas[7]}, ignore_index=True)


,Loss,Accuracy,Precision,Recall,F1,Specificity,fpr,fnr,AUC
Número de neuronas,,,,,,,,,
512.0,0.16,0.61,0.73,0.75,0.74,0.24,0.76,0.25,0.49
1024.0,0.14,0.60,0.73,0.73,0.73,0.26,0.74,0.27,0.50


In [ ]:
#Simple3 AlexNet
import pandas as pd
from keras.callbacks import EarlyStopping

def neuronas(num_neuronas, epochs, ruta, batch_size):

    '''
    Función que devuelve una tabla comparativa para distintas valores de neuronas introducidos como parámetros a partir del modelo y el batch size
    seleccionado previamente.
    ------------------------------------------------------------------------
    Parámetros;
    - num_neuronas:
    - epochs:
    - ruta: str. Ruta base donde se encuentran las imágenes organizadas en subcarpetas (train, val, test)
    - batch_size: int. Tamaño del lote que se utiliza en una única iteración del algoritmo de aprendizaje. Se emplea dentro de la función
    "preparar_modelo" para determinar el tamaño del lote para cada uno de los generadores (train, val y test)
    ----------------------------------------------------------------
    Return:
    - compara_neuronas_def: dataframe que contiene como índice las columnas referidas al número de neuronas. El dataframe 
    obtenido se observa como una tabla comparativa de diversas métricas para cada número de neuronas.
    '''
    
    #se inicializa un dataframe vacío donde, posteriormente se van a añadir todos los componentes necesarios para comparar y determinar cual es el mejor
    #valor de neuronas en la capa oculta
    compara_neuronas=pd.DataFrame()
    
    input_shape=(340,340,3)

    #se emplea la función preparar_modelo para configurar los generadores de datos para entrenar, validar y probar 
    #un modelo de aprendizaje automático con imágenes
    train_generator, validation_generator, test_generator = preparar_modelo(ruta, batch_size)
    
    
    for neurona in num_neuronas:
        print(f"Modelo con {neurona} neuronas en su capa oculta...")
        #se emplea el modelo Simple2 que es el que se ha determinado previamente como "mejor"
        model = keras.Sequential(
            [
                keras.Input(shape=input_shape),
                layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu'),
                layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'),
                layers.BatchNormalization(),
                
                layers.Flatten(), #convierte imágenes en vectores
                layers.Dense(neurona, activation="relu"), 
                layers.Dropout(0.2),
                layers.Dense(neurona, activation="relu"), 
                layers.Dropout(0.2),
                layers.Dense(1, activation="sigmoid"), #produce una probabilidad entre 0 y 1 para la clasificación binaria
            ]
        )

        
        #se compila el modelo y se calculan las métricas con las que se quiere trabajar
        #en este caso, en la función de pérdida "loss", se emplea la entropía cruzada binaria "binary_crossentropy" ya que se trata de 
        #un problema de clasificación binaria
        model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy","Recall","AUC"]) #cambias loss
    
        #ENTRENA
        # con callbacks se detiene el entrenamiento si la pérdida en el conjunto de validación no mejora después de 10 épocas (patience)
        #se emplea un batch size de 32 que es el que ha dado mejores resultados antes
        model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=EarlyStopping(monitor='val_auc', patience=10,restore_best_weights=True))
    
        #se calculan las métricas
        y_test=test_generator.labels
        y_pred=model.predict(test_generator)
        calculo_metricas=metricas(y_test, y_pred) #se llama a la función creada previamente para calcular las métricas de cada modelo
        #se calcula loss a partir de la evaluación del modelo
        loss=model.evaluate(test_generator, verbose=0)[0]
        #esto es en caso de querer meter todos estos parametros dentro de metricas (cambiando tambien la linea de arriba, en lugar de metricas loss, accuracy...)
        #metricas = f"Loss: {loss}, Accuracy: {accuracy}, Recall: {recall}, AUC: {AUC}, Precision: {precision}"
        #cambiar .append por .concat
        #se añaden todos los componentes necesarios para comparar los distintos modelos de arquitectura para distintos batch size 
        #(comparando las métricas)
        compara_neuronas=compara_neuronas.append({"Número de neuronas": int(neurona), "Loss": loss, "Accuracy": calculo_metricas[0], "Precision": calculo_metricas[1], "Recall": calculo_metricas[2], "F1":calculo_metricas[3], "Specificity":calculo_metricas[4], "fpr":calculo_metricas[5], "fnr":calculo_metricas[6], "AUC": calculo_metricas[7]}, ignore_index=True)
    
    #se fija la columna "Número de neuronas" como índice. 
    compara_neuronas.set_index("Número de neuronas", inplace=True) #inplace=True se pone para modificar el dataframe original ya que sino, no se modifica
    compara_neuronas_def = compara_neuronas.round(2) #se redondean los decimales a 2
    #compara_neuronas_def['Número de neuronas'] = compara_neuronas_def['Número de neuronas'].astype(int) #para convertr la columna Numero neuronas a entero y no aparezca como decimales
    return compara_neuronas_def
    
        #PONER int(neurona) si salen como decimanles o poner lo que esta comentado

In [ ]:
num_neuronas=[512, 1024, 2048] #lista con distintos valores de neuronas para probar
epochs=20
ruta='C:/Users/nuria/Downloads/TFG/data_nuevo'
batch_size=32

neuronas(num_neuronas, epochs, ruta, batch_size)

Por lo tanto, se puede apreciar que, el mejor modelo se corresponde con 64 neuronas en la capa oculta ya que, 
tiene un valor mayor en la gran parte de métricas (aunque en loss deberia ser menor) CAMBIAR EN CASO NECESARIO

## Matriz de confusión para ver como funciona el modelo elegido finalmente

Finalmente, se obtiene la matriz de confusión para el modelo final obtenido.

In [ ]:
#CAMBIAR TODO LO QUE HAY A CONTINUACION POR EL MODELO FINAL

In [ ]:
ruta='C:/Users/nuria/Downloads/TFG/data_nuevo'
batchsize=20
preparar_modelo(ruta, batch_size)

In [ ]:
#se trabaja con el modelo simple1
input_shape=(150,150,3)

model = keras.Sequential( #funcion establecer arquitectura(simple...)
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"), 
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        #se necesitan mas capas
        layers.Dropout(0.5), #probar otros valores (este es muy alto)
        layers.Dense(1, activation="sigmoid"), #una unica neurina, sigmoide
    ]
)

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping


epochs = 20

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy","Recall","AUC"]) #cambias loss

# con callbacks se detiene el entrenamiento si la pérdida en el conjunto de validación no mejora después de 10 épocas (patience)
model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=EarlyStopping(monitor='val_auc', patience=10,restore_best_weights=True)) 

In [ ]:
y_test=test_generator.labels
y_pred=model.predict(test_generator)

In [ ]:
y_pred=y_pred>0.5 #para convertirlo en un problema binario

In [ ]:
#matriz de confusión con sklearn
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred) #.ravel()

In [ ]:
matriz

In [ ]:
#PERCEPTRON SKLEARN

import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np 

labels=np.unique(y_test)

matriz_conf = metrics.confusion_matrix(y_test, y_pred,labels=labels)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = matriz_conf, display_labels = ["neumonía" , "no neumonía"])
fig, ax = plt.subplots(figsize=(5,5))
cm_display.plot(ax=ax)
plt.title("Matriz de confusión neumonía-no neumonía")
plt.show()

Se puede comprobar como han mejorado los resultados respecto al modelo más simple ya que...

In [ ]:
#ARTICULO METIDO EN EL RESUMEN DE LA MEMORIA
#https://www.sciencedirect.com/science/article/pii/S001048252030247X?via%3Dihub#bib1